In [1]:
from sqlalchemy import create_engine
import pandas as pd
import json
from MyCreds.mycreds import Capstone_Local       #from local site-packages

engine = create_engine(f'postgresql+psycopg2://{Capstone_Local.username}:{Capstone_Local.password}@{Capstone_Local.host}/postgres', echo=False)

sql_alc_cnxn = engine.connect()

# Load Data into local DB

We use the postal code database to do the set-up

In [2]:
# load data from JSON
postal = 'buildings.json'
df_postal = pd.read_json(postal)

# upload to AWS
df_postal.to_sql('postal',sql_alc_cnxn, if_exists='replace', index=False)

In [2]:
# check to ensure it is set up right
sql_query ='''
select * from public."postal";
'''
df = pd.read_sql(sql_query, sql_alc_cnxn)
df.sample()

,ADDRESS,BLK_NO,BUILDING,LATITUDE,LONGITUDE,LONGTITUDE,POSTAL,ROAD_NAME,SEARCHVAL,X,Y,location
122208,475 CHOA CHU KANG AVENUE 3 KITS4KIDS SPECIAL S...,475,KITS4KIDS SPECIAL SCHOOL,1.377968,103.739176,103.739176,680475,CHOA CHU KANG AVENUE 3,KITS4KIDS SPECIAL SCHOOL,17523.064547,39994.462596,0101000020110F0000705E8F8C280CF63F0DCB73A74EEF...


# Setting up PostGIS in AWS

We follow steps from [this documentation](https://docs.aws.amazon.com/AmazonRDS/latest/UserGuide/Appendix.PostgreSQL.CommonDBATasks.PostGIS.html) to setup/activate the PostGIS extention

Run queries in PGAdmin

Ensure that the rds_superuser role (user name that was used to create the DB instance) is selected

# Next set up a spatial table in the database

2 Main Steps
1. Create a normal non-spatial table/use existing 
2. Add a spatial column to the table using the OpenGIS "AddGeometryColumn" function.

But we since we have lat-long in the database we can just use that


In [3]:
sql_query ='''
ALTER TABLE public."postal" ADD COLUMN location GEOMETRY(point, 3857);
UPDATE public."postal" SET location = ST_SETSRID(ST_MakePoint("LATITUDE","LONGITUDE"),3857);
'''
sql_alc_cnxn.execute(sql_query)


Check to ensure location was loaded

In [ ]:
sql_query ='''
select * from public."postal";
'''
df = pd.read_sql(sql_query, sql_alc_cnxn)
df

,ADDRESS,BLK_NO,BUILDING,LATITUDE,LONGITUDE,LONGTITUDE,POSTAL,ROAD_NAME,SEARCHVAL,X,Y,location
0,101A BAYFRONT AVENUE TEMPORARY SITE OFFICE SIN...,101A,TEMPORARY SITE OFFICE,1.275697,103.855652,103.855652,18895,BAYFRONT AVENUE,TEMPORARY SITE OFFICE,30485.511362,28685.612665,0101000020110F00004BC54D264169F43FC685CAFFC2F6...
1,1 STRAITS BOULEVARD SINGAPORE CHINESE CULTURAL...,1,SINGAPORE CHINESE CULTURAL CENTRE,1.275829,103.849576,103.849576,18906,STRAITS BOULEVARD,SINGAPORE CHINESE CULTURAL CENTRE,29809.365407,28700.236127,0101000020110F000013AF0BDFCB69F43F8F26A5755FF6...
2,11A STRAITS BOULEVARD TEMPORARY SITE OFFICE SI...,11A,TEMPORARY SITE OFFICE,1.274950,103.851665,103.851665,18907,STRAITS BOULEVARD,TEMPORARY SITE OFFICE,30041.838898,28602.987244,0101000020110F00007A735AA63166F43F37F1EDAE81F6...
3,2 CENTRAL BOULEVARD IOI CENTRAL BOULEVARD TOWE...,2,IOI CENTRAL BOULEVARD TOWERS,1.279777,103.851513,103.851513,18916,CENTRAL BOULEVARD,IOI CENTRAL BOULEVARD TOWERS,30024.919852,29136.807026,0101000020110F0000960CD0D6F779F43FB11B70317FF6...
4,21 PARK STREET DBS MARINA BAY MRT STATION SING...,21,DBS MARINA BAY MRT STATION,1.276427,103.854598,103.854598,18925,PARK STREET,DBS MARINA BAY MRT STATION,30368.205612,28766.381902,0101000020110F0000AA1D07173F6CF43F2399DFBAB1F6...
...,...,...,...,...,...,...,...,...,...,...,...,...
142881,100A KRANJI LOOP SINGAPORE 887327,100A,NIL,1.433629,103.758648,103.758648,887327,KRANJI LOOP,100A KRANJI LOOP SINGAPORE 887327,19690.280996,46149.118386,0101000020110F00008F5DBB7B25F0F63FDCF8F2AF8DF0...
142882,A PASIR PANJANG ROAD SINGAPORE 887328,A,NIL,1.277170,103.795840,103.795840,887328,PASIR PANJANG ROAD,A PASIR PANJANG ROAD SINGAPORE 887328,23828.902741,28848.553424,0101000020110F0000E717771F4A6FF43F7788C209EFF2...
142883,PULAU BUKOM SINGAPORE 903808,,NIL,1.235596,103.768645,103.768645,903808,PULAU BUKOM,PULAU BUKOM SINGAPORE 903808,20802.263821,24251.557450,0101000020110F0000B486DF7800C5F33F19D2057C31F1...
142884,GATE C7 AIRPORT CARGO ROAD CHANGI ANIMAL AND P...,GATE C7,CHANGI ANIMAL AND PLANT QUARANTINE CENTRE,1.375285,103.996737,103.996737,918104,AIRPORT CARGO ROAD,CHANGI ANIMAL AND PLANT QUARANTINE CENTRE,46186.488419,39698.194002,0101000020110F00005B6A881C2B01F63FB09F7E8ACAFF...


In [ ]:
# some test points from google maps
CBD = ('1.2790515225747496', '103.85402179404132')
Tiong_Bahru = ('1.2863988893455323', '103.82509579340865')

# Test spatial queries

Next we can query distances to popular locations in Singapore to each address location

Note ST_Distance is returning in degrees so we have to x111239.6925397 to convert this to meters

In [ ]:
sql_query ='''
SELECT
    "ADDRESS",
    ST_Distance(
            "location",
            ST_GeomFromText('POINT(1.279177928627326 103.85462727093415)',3857)
            )*111239.6825397 as m_to_CBD,
    ST_Distance(
            "location",
            ST_GeomFromText('POINT(1.2863988893455323 103.82509579340865)',3857)
            )*111239.6825397 as m_to_tiongbahru
    FROM public."postal";
'''
df = pd.read_sql(sql_query, sql_alc_cnxn)
df.sample(20)

,ADDRESS,m_to_cbd,m_to_tiongbahru
70380,3 EVERGREEN GARDENS SINGAPORE 468879,9713.310132,12557.423144
41732,1A MELROSE DRIVE BRADDELL HEIGHTS ESTATE SINGA...,7722.773119,8129.366010
44233,34 CIRCUIT ROAD SINGAPORE 370034,6640.118674,8402.190448
135378,31 FERNVALE CLOSE RIVERTREES RESIDENCES SINGAP...,13025.309484,13412.522608
22847,1B STEVENS DRIVE SINGAPORE 257890,5268.058986,3414.493372
134556,61 LENTOR GROVE FUDU PARK SINGAPORE 789221,12152.648911,11307.015630
61350,722 DUNMAN ROAD ZEE JUNIORS SINGAPORE 439235,5965.225549,8608.430992
48916,1077 EUNOS AVENUE 6 EUNOS INDUSTRIAL ESTATE SI...,6559.330693,8750.832509
40779,83 GENTING LANE GENTING BUILDING SINGAPORE 349568,5825.856170,7272.093142
110228,24 TOH GUAN ROAD EAST WESTLITE TOH GUAN DORMIT...,13060.235806,9837.076851


Alternatively, instead of creating the point, we can also use the function ST_MakePoint() to create the point from latitude and longitude columns

In [ ]:
sql_query ='''
SELECT
    "ADDRESS",
    ST_Distance(
            ST_SETSRID(ST_MakePoint("LATITUDE","LONGITUDE"),3857),
            ST_GeomFromText('POINT(1.279177928627326 103.85462727093415)',3857)
            )*111239.6825397 as m_to_CBD,
    ST_Distance(
            ST_SETSRID(ST_MakePoint("LATITUDE","LONGITUDE"),3857),
            ST_GeomFromText('POINT(1.2863988893455323 103.82509579340865)',3857)
            )*111239.6825397 as m_to_tiongbahru
    FROM public."postal";
'''
df = pd.read_sql(sql_query, sql_alc_cnxn)
df.sample(20)

,ADDRESS,m_to_cbd,m_to_tiongbahru
85412,3B CHARLTON ROAD CHARLTON RESIDENCES SINGAPORE...,9028.189551,9932.231249
109857,38 TOH TUCK PLACE TOH TUCK GARDEN SINGAPORE 59...,11895.874264,8791.208420
51619,276 CHANGI ROAD SINGAPORE 419756,7159.096469,9672.787595
110999,2A SECOND CHIN BEE ROAD SINGAPORE 618781,16586.391865,13234.294866
64632,31 JALAN BULOH PERINDU SEASIDE PARK SINGAPORE ...,8028.480395,10908.218289
31127,40 WATTEN VIEW WATTEN HILL SINGAPORE 287167,7643.521987,5241.852854
109182,535 CLEMENTI ROAD NGEE ANN POLYTECHNIC (BLK 43...,10995.906297,7877.946668
81442,23 THRIFT DRIVE SINGAPORE 535032,7806.936276,9077.128907
81957,221 AIRPORT ROAD PAYA LEBAR AIR BASE SINGAPORE...,8786.603323,10511.354880
138613,41 SUNRISE CLOSE SUNRISE VILLA SINGAPORE 806638,11994.563856,11701.148855


## Sanity Checking

Results look more or less inline with what we can get from a good map - precision is off as the selected points on the map are not precise

![query_results](query_res.png)
![dist_to_cbd](dist_to_cbd.png)
![dist_to_tiongbahru](dist_to_tiongbahru.png)

## Load MRT/LRT Data

In [14]:
df_transit = pd.read_csv('lta-mrt-station-exit/mrt_lrt_data.csv')

In [16]:
df_transit.to_sql('transit',sql_alc_cnxn, if_exists='replace', index=False)

In [7]:
sql_query ='''
ALTER TABLE public."transit" ADD COLUMN location GEOMETRY(point, 3857);
UPDATE public."transit" SET location = ST_SETSRID(ST_MakePoint("lat","lng"),3857);
'''
sql_alc_cnxn.execute(sql_query)

In [8]:
sql_query ='''
select * from public."transit";
'''
df = pd.read_sql(sql_query, sql_alc_cnxn)
df

,station_name,type,lat,lng,location
0,Jurong East,MRT,1.333207,103.742308,0101000020110F0000B9C5FCDCD054F53FC9E369F981EF...
1,Bukit Batok,MRT,1.349069,103.749596,0101000020110F000040C23060C995F53F984D8061F9EF...
2,Bukit Gombak,MRT,1.359043,103.751863,0101000020110F000041B96DDFA3BEF53FA304FD851EF0...
3,Choa Chu Kang,MRT,1.385417,103.744316,0101000020110F0000AE282504AB2AF63FEE7893DFA2EF...
4,Yew Tee,MRT,1.397383,103.747523,0101000020110F00002AC8CF46AE5BF63F8080B56AD7EF...
...,...,...,...,...,...
152,Punggol Point,LRT,1.416932,103.906680,0101000020110F0000747E8AE3C0ABF63FFCFB8C0B07FA...
153,Samudera,LRT,1.415955,103.902185,0101000020110F00000FB9196EC0A7F63F467C2766BDF9...
154,Nibong,LRT,1.411865,103.900321,0101000020110F000065DF15C1FF96F63FBBB9F8DB9EF9...
155,Sumang,LRT,1.408501,103.898605,0101000020110F0000FB2136583889F63F6AC18BBE82F9...


In [27]:
sql_query ='''
SELECT
    a."ADDRESS",
    count(b.*) as station_count
    FROM public."postal" a, public."transit" b
    WHERE ST_DWithin(a."location",b."location",600/111239.6825397) 
    GROUP BY a."ADDRESS" 
'''
df = pd.read_sql(sql_query, sql_alc_cnxn)
df.sample(20)

,ADDRESS,station_count
16133,22 JALAN NOVENA RESIDENCES @ NOVENA SINGAPORE ...,1
12107,1A LORONG SARINA SINGAPORE 416646,1
34141,49 KIM PONG ROAD SINGAPORE 160049,1
30003,406 DUNEARN ROAD WATTEN ESTATE SINGAPORE 289611,2
25450,33 EVERTON ROAD BLAIR PLAIN CONSERVATION AREA ...,1
17129,23 SAINT THOMAS WALK SAM KIANG MANSIONS SINGAP...,1
6534,146 MACKENZIE ROAD SINGAPORE 228723,1
18679,260 BEDOK ROAD SIEW LIM PARK SINGAPORE 469427,1
6471,145A TYRWHITT ROAD PETAIN ROAD/TYRWHITT ROAD C...,3
24466,323 ANG MO KIO AVENUE 3 CHENG SAN FAMILY SERVI...,1


In [28]:
df[df['ADDRESS'].str.contains('COMPASSVALE CRESCENT')]

,ADDRESS,station_count
20901,290A COMPASSVALE CRESCENT COMPASSVALE LODGE SI...,1
20902,290A COMPASSVALE CRESCENT PCF SPARKLETOTS PRES...,1
20909,290 COMPASSVALE CRESCENT MULTI STOREY CAR PARK...,1
20950,293A COMPASSVALE CRESCENT COMPASSVALE BOARDWAL...,1
20955,293B COMPASSVALE CRESCENT COMPASSVALE BOARDWAL...,1
20957,293 COMPASSVALE CRESCENT COMPASSVALE BOARDWALK...,1
20958,293D COMPASSVALE CRESCENT COMPASSVALE BOARDWAL...,1
20968,294A COMPASSVALE CRESCENT COMPASSVALE BOARDWAL...,1
20973,294 COMPASSVALE CRESCENT SINGAPORE 540294,1
